# Generalzacion problema de particiones:

Ahora vamos a resolver un problema un poco mas general. Tenemos el conjunto $S = \{a_1, \dots, a_N\}$ con $N$ elementos, y lo queremos particionar en conjuntos $S_1, S_2, \dots, S_m$ tales que la suma en $S_i$ sea menor a un peso maximo $P_i$

$$\sum_{a \in S_i} a \leq P_i$$

con $i = 1, 2, \dots, m$

Las variables que usaremos seran:

$$x_{i}^{k} = \left\{ \begin{array}{lcc}
             1 & si & a_i \in S_k \\
             \\ 0 & si & a_i \notin S_k \\
             \end{array}
   \right.$$

Es decir, tenemos $m$ variables por cada uno de los $N$ elementos de $S$. La restriccion de peso se escribe como

$$\sum_{i = 1}^{N} a_ix_i^{k} \leq P_k$$

Esta parte de la restriccion se representara de forma usual, usando variables de slack.


### Representacion de particion:

Queremos que los conjuntos $S_i$ particionen $S$, para esto, agregamos un sumando 

$$\left[ \sum_{k = 1}^{m} x_i^{k} - 1\right]^2$$

Que penaliza si el elemento $i$ esta en mas de un conjunto, o no esta en ningun conjunto

De forma matricial, la expresamos como:

 \begin{pmatrix}
    -1 & 1 & \dots & 1 \\
    1 & -1 & \dots & 1 \\
    \vdots & \vdots & \ddots & \vdots \\
    1 & 1 & \dots & -1
  \end{pmatrix}_{m \times m}

Es decir, una matriz de $m \times m$ con $-1$ en su diagonal principal y $1$ en el resto. Esta restriccion la usaremos para todos los elementos de $S$, asi, la matriz $Q_{particion}$ que penaliza si los conjuntos $S_i$ no particionan $S$ se calcula como:

$$Q_{particion} = I_N \bigotimes  \begin{pmatrix}
    -1 & 1 & \dots & 1 \\
    1 & -1 & \dots & 1 \\
    \vdots & \vdots & \ddots & \vdots \\
    1 & 1 & \dots & -1
  \end{pmatrix}_{m \times m}$$

Donde $I_N$ es la matriz identidad de $N \times N$ y $\otimes$ representa el producto de Kronecker 



In [ ]:
import os
import time
import numpy as np
import pandas as pd
import math
from utils import *
import dimod
import warnings
import matplotlib.pyplot as plt
from functions_dwave import *
warnings.filterwarnings('ignore')

In [ ]:
# Creamos una instancia del problema
set = np.array([3, 1, 1, 2, 2, 1])
set = np.array([7, 15, 3, 2, 5, 5, 10, 7])
set = np.array([11, 5, 1, 5]) # Este array solo puede ser particionado de una forma
set = np.array([1, 1, 1, 1, 1, 5, 10])
set = np.array([1, 2, 3, 4])
pesos = np.array([3, 4, 3]) # Pesos maximos en cada conjunto
#set = np.array([1, 2, 3, 4, 10])

set = np.array([1, 2, 3, 4, 5, 5])
pesos = np.array([10, 5, 5])
suma = sum(set)
size = len(set)
conjuntos = len(pesos) # EN CUANTOS CONJUNTOS PARTICIONAMOS S
data = dict({'set': set})
df = pd.DataFrame(data)
print("-------------------------------------")
print("Partition the set: \n ")
print(df)
print("With total sum of: ", suma)

### Creamos la matriz particion:

La calculamos como se comento arriba, usando el producto de Kronecker. Vale la pena notar que esta estructura de la matriz se realiza para obtener un vector solucion de la forma:

$$v = [x_1^1, \dots, x_1^m, x_2^1, \dots, x_2^m, \dots, x_N^1, \dots, x_N^m]

In [ ]:
# Matriz particion:
q_particion = np.ones((conjuntos, conjuntos)) - 2 * np.identity(conjuntos)
q_particion = np.kron(np.identity(size), q_particion) 
# Le tendremos que agregar ceros al final, cuando agregemos las restricciones

print(q_particion)

### Creamos la matriz de restriccion:

Es la que va a agregar una penalizacion en caso de que la suma en $S_k$ sea mayor que $P_k$

$$\sum_{i = 1}^{N} a_ix_i^k \leq P_k$$

para esto usamos variables slacks de la forma usual para cada $k = 1, \dots, m$.

#

Los slacks los organizaremos en el vector solucion de la siguiente forma:

$$sol = \left[\overrightarrow{v}, \overrightarrow{s_1}, \dots, \overrightarrow{s_m}\right]$$

donde $\overrightarrow{v}$ es el vector solucion descrito anteriormente, y $\overrightarrow{s_k}$ el vector con los coeficientes de las variables de slacks para la restriccion de peso maximo en $S_k$.

$$ \overrightarrow{s_k} = [2^0, 2^1, \dots, 2^{L_k - 2}, P_k + 1 - 2^{L_k - 1}]$$

siendo $L_k = \lfloor log_2(P_k) \rfloor + 1$ la cantidad de variables de slack a usar.


### Calculo de la matriz de restriccion para $S_k$

La matriz de restriccion para $S_k$, que llamaremos $Q_k$, se calcula como:

$$Q_k = (sol_k)^t(sol_k) - 2P_kD$$

con $D$ la matriz diagonal con diagonal $sol_k$ y 

$$sol_k = [0_1^1, \dots, 0_1^{k-1}, a_1, 0_1^{k+1}, \dots, 0_1^m, \dots, 0_N^1, \dots, 0_N^{k-1}, \dots, a_N, 0_N^{k+1}, \dots, 0_N^m, 
            0\overrightarrow{s_1}, \dots, 0\overrightarrow{s_{k-1}}, \overrightarrow{s_k}, 0\overrightarrow{s_{k+1}}, \dots, 0\overrightarrow{s_m}]$$

Es decir $sol_k$ es el vector $sol$ "activando" unicamente los elementos correspondientes al conjunto $S_k$ (tanto las $x_i^k$ como las slacks).

Finalmente, hallamos $Q_{restriccion}$ como:

$$Q_{restriccion} = \sum_{k = 1}^{m} Q_k$$



In [ ]:
# Matriz de restriccion: 
vect_S = set

# Debemos agregar a vect los pesos de cada slack:
num_slacks = np.floor(np.log2(pesos)) + np.ones(conjuntos)
dimensiones = int(size*conjuntos + sum(num_slacks))

# Revisar esto, lo mejor seria no usar un for:
# Igualmente, el for agrega una cantidad de iteraciones igual a la cantidad de conjuntos, 
# Si la cantidad es razonable, no habra un degradamiento en el rendimiento
n = 0
pesosSlacks = []
q_restriccion = np.zeros((dimensiones, dimensiones))
for num_slack in num_slacks:
    pesosSlacks = np.r_[(2 ** (np.arange(num_slack - 1))).astype(float)]
    pesosSlacks = np.r_[np.array(pesosSlacks), pesos[n] + 1 - 2**(num_slack - 1)]
    
    antes = np.zeros(int(sum(np.array(num_slacks)[0:n])))
    desp = np.zeros(int(sum(np.array(num_slacks)[n+1:len(num_slacks)])))

    # New nums es igual a set, agregando ceros entre los elementos
    new_nums = np.zeros(size + (size-1)*(conjuntos-1))
    new_nums[::conjuntos] = vect_S
    new_nums = np.r_[new_nums, np.array(np.zeros(conjuntos-1))]

    # Rota los elementos en new_nums para que los elementos no nulos coincidan con el conjunto para el cual
    # queremos representar la restriccion
    new_nums = np.roll(new_nums, n)
    vect = np.r_[new_nums, antes, pesosSlacks, desp]
    print(vect)

    q_restriccion = q_restriccion + np.outer(vect, vect) - 2 * pesos[n] * np.diag(vect)

    n = n+1


# La dimension de q_particion es de size*conjuntos. Agregamos ceros al final para que la matriz llegue 
# a un tamaño de "dimensiones"
q_particion = np.pad(q_particion, [(0, dimensiones - size*conjuntos), (0, dimensiones - size*conjuntos)], mode='constant')
    


In [ ]:
#Q objetivo:

alpha = 10
beta = 1
qubo = alpha * q_particion + beta * q_restriccion

In [ ]:
# Enviamos problema a Dwave:
print("simmulating....")
sampleset = sendToDwave(qubo, 100, True) # aggregate = True
print("Filtering:")
# Nos quedamos con la solucion que minimiza la energia:
energies = [element[1] for element in sampleset]
solution = sampleset[energies.index(min(energies))][0] # Le saco la energia

print(sampleset)
print(solution[0:size*conjuntos])

plt.bar([(str(el[1])) for el in sampleset], [(el[1]) for el in sampleset])
plt.xticks(rotation = 90)
plt.show()